## Similarities in Dynamical Systems of Human Brain Resting States Inferred from Human Connectome Project MEG Data

Here, we demonstrate the similarities between inferred human brain resting state networks across various patients found in the Human Connectome Project (HCP) MEG2 data release. 

Resting state connectivities between MEG channels in sensor space are inferred and further analyzed to discover any commonalities between groups of patients found in the HCP database. 

The necessary packages are first imported to the Jupyter notebook **below:**

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats.mstats import linregress
%matplotlib inline

### Importing the Inference Code
The **main** function for the inference script is also imported to the notebook, which works to generate all the connectivities and time shift covariances between channels for every given patient in the HCP database. 

Entry i, j **connectivities (W) matrix** can be thought of as weights describing the effect that the ith channel has on jth channel of the MEG machine used to record data. 

Entry i, j in the **time shift covariances matrix**  can be thought of as the covariance between the ith channel and the jth channel at a given timeshift. This given timeshift is unitless, and depends on the sampling frequency given by the MEG recording. 

In [2]:
import RestingStateMEG_HCP_Inference

### Setting the parameters for running the script
We must first set the parameters for the script, which includes the list of identifiers for the specific patients within the HCP database that we would like to analyze, the number of PCA components (principal MEG channels) used for the inference, and the size of the time shift for the generation of time shift covariances between the MEG channels. 

Below, all 88 patients' identifiers are listed for processing

In [3]:
pca_components = 10
time_shift = 10
dataset_ids = ['100307', '102816', '105923', '106521', '108323', '109123', '111514', '112920', 
               '116524', '116726', '133019', '140117', '146129', '149741', '153732', '154532', 
               '156334', '158136', '162026', '162935', '164636', '166438', '169040', '172029', 
               '174841', '175237', '175540', '177746', '179245', '181232', '185442', '187547', 
               '189349', '191033', '191437', '191841', '192641', '195041', '198653', '204521', 
               '205119', '212318', '212823', '214524', '221319', '223929', '233326', '248339', 
               '250427', '255639', '257845', '283543', '287248', '293748', '352132', '352738', 
               '353740', '358144', '406836', '433839', '512835', '555348', '559053', '568963', 
               '581450', '599671', '601127', '660951', '662551', '665254', '667056', '679770', 
               '680957', '706040', '707749', '715950', '725751', '735148', '783462', '814649',
               '825048', '872764', '877168', '891667', '898176', '912447', '917255', '990366']

### Important Variables 
We can finally run the main script, which will produce five variables needed for further analysis of the inference on the MEG data

1. allwmatrices
2. allsigs
3. allbiases
4. pca_data_dict

**allwmatrices** contains all the w matrices for the given dataset of patients by the user. **allsigs** and **allbiases** contains the sigma and bias values recovered from the inference function which are needed to simulate data and help to prove the predictive capabilities of the inference function. Lastly **pca_data_dict** is a dictionary that contains the transformed data from the Principal Components Analysis on the original MEG data with patient identifiers as keys. 

In [ ]:
allwmatrices, allsigs, allbiases, pca_data_dict = RestingStateMEG_HCP_Inference.hcp_inference(dataset_ids, pca_components, time_shift)

### Testing the Predictive Power of the Network Inference
After these 4 variables are collected, we can then proceed to plot and analyze the results of the inference on the MEG dataset. To test the predictive capabilities of the W matrix formed from the inference for each patient, we can compare the time shift covariance of the MEG recording (that has already undergone a PCA transformation with the number of components given by the user) with the time shift covariance of a simulated dataset (which is simulated using the same W matrix computed from the MEG recording). 

If the W matrix is predictive, we would expect to see a very high correlation within the plots between the time shift covariance matrices, as this means that the simulated data from the W matrix exhibits very similar time shift relationships between MEG channels as the non-simulated MEG recording

In [ ]:
# allwmatrices is a dictionary with patient identifiers as keys. allwmatrices[key] will return the w matrix 
# for that specific patient

fig = plt.figure()

plot_counter = 1
for patient in allwmatrices:
    w = allwmatrices[patient]
    sig = allsigs[patient]
    bias = allbiases[patient]
    
    x_original = pca_data_dict[patient].T
    
    # simulate MEG timecourse data using the w, sigma, and bias computed from the inference code
    simulated_data = RestingStateMEG_HCP_Inference.simulate(pca_components, 1018, w=w, sigma=sig, rho=0, bias=bias)
    
    # compute the time shift covariance matrices for the 'original' (transformed PCA MEG timecourse) and simulated data
    tsc_original = RestingStateMEG_HCP_Inference.time_shift_cov(x_original, shift=10)
    tsc_simulated = RestingStateMEG_HCP_Inference.time_shift_cov(simulated_data[0], shift=time_shift)
    
    plt.subplot2grid((22,4),(int(plot_counter/4),plot_counter%4))
    plt.title('Patient ' + str(patient))
    plt.xlabel('time_shift_cov original')
    plt.ylabel('time_shift_cov simulated')
    plt.plot(tsc_original, tsc_simulated, 'ko', alpha=0.5)
    plt.plot([-1, 1], [-1,1], 'r--')
    plt.show()
    

plt.show()



### Clustering of W matrices

Interestingly, a spectral clustering analysis of all the computed W matrices for the patients in the HCP database leads to the formation of several groups of similar W matrices, indicating shared resting state brain networks among different patients. 

In [ ]:
# spectral clustering, spectrally cluster the W matrices of all the patients to see if certain patients fall into 
# certain categories. Do for all 88 patients

def wmatrix_clustering_prepare(allwmatrices):
    flattened_wmatrix_array = []
    unflattened_wmatrix_dict = {}
    counter = 0
    for patient in allwmatrices:
        flattened_matrix = allwmatrices[patient].flatten()
        unflattened_wmatrix_dict[counter] = allwmatrices[patient]
        if len(flattened_wmatrix_array) == 0:
            flattened_wmatrix_array = [flattened_matrix]
        else:
            flattened_wmatrix_array.append(flattened_matrix)
        counter = counter + 1
    return flattened_wmatrix_array, unflattened_wmatrix_dict

flattened_wmatrix_array, unflattened_wmatrix_dict = wmatrix_clustering_prepare(allwmatrices)
print(np.array(flattened_wmatrix_array).shape)
print(unflattened_wmatrix_dict)